# Split data by Stadsdeel

In [ ]:
%matplotlib widget
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
import shapely.geometry as sg
import pathlib

import set_path
import upc_sw.poly_utils as poly_utils
import upc_sw.scraping_utils as scraping

In [ ]:
district_dict = {'B': 'haven',
                 'T': 'zuid_oost',
                 'A': 'centrum',
                 'K': 'zuid',
                 'E': 'west',
                 'M': 'oost',
                 'N': 'noord',
                 'F': 'nieuw_west'}

In [ ]:
cyclomedia_dict = {'nl-amsd-200519-7415-laz': 'haven',
                   'nl-amsd-200604-7415-laz': 'zuid_oost',
                   'nl-amsd-200605-7415-laz': 'ring',
                   'nl-amsd-200606-7415-laz': 'centrum',
                   'nl-amsd-200608-7415-laz': 'zuid',
                   'nl-amsd-200618-7415-laz': 'west',
                   'nl-amsd-200622-7415-laz': 'oost',
                   'nl-amsd-200702-7415-laz': 'noord',
                   'nl-amsd-200707-7415-laz': 'nieuw_west',
                   'nl-amsd-200823-7415-laz': 'haven',
                   'nl-amsd-200824-7415-laz': 'west',
                   'nl-amsd-200903-7415-laz': 'zuid',
                   'nl-amsd-200904-7415-laz': 'ring',
                   'nl-amsd-200918-7415-laz': 'noord',
                   'nl-amsd-200920-7415-laz': 'nieuw_west',
                   'nl-amsd-200921-7415-laz': 'centrum',
                   'nl-amsd-200923-7415-laz': 'zuid_oost',
                   'nl-amsv-201001-7415-laz': 'oost'}

In [ ]:
# Stadsdelen dataset
stadsdelen_folder = '../../datasets/Stadsdelen/'

for (_, stdsdl) in district_dict.items():
    pathlib.Path(f'{stadsdelen_folder}{stdsdl}').mkdir(parents=True, exist_ok=True)

## Generate polygons for each stadsdeel

In [ ]:
# Load neighbourhood data and convert to RD
neighbourhood_data = '../datasets/neighbourhoods_amsterdam.csv'

neighbourhood_gdf = pd.read_csv(neighbourhood_data, index_col=0)
neighbourhood_gdf['geometry'] = gpd.GeoSeries.from_wkt(neighbourhood_gdf['geometry'])
neighbourhood_gdf = gpd.GeoDataFrame(neighbourhood_gdf, crs='wgs84')
neighbourhood_gdf.to_crs(epsg='28992', inplace=True)

In [ ]:
neighbourhood_gdf['naam'] = neighbourhood_gdf['district_code'].replace(district_dict)
stadsdelen_poly = neighbourhood_gdf[['naam', 'geometry']].dissolve(by='naam')

In [ ]:
stadsdelen_poly.to_file('../../datasets/Stadsdelen/stadsdelen.gpkg', driver='GPKG')

## Generate polygons for all CycloMedia tiles epr stadsdeel

In [ ]:
# Load tile data
tiles_file = f'../datasets/tile_info/AMS_run2_tiles_list.csv'
all_tiles_gdf = pd.read_csv(tiles_file)
all_tiles_gdf['geometry'] = all_tiles_gdf.apply(
    lambda row: sg.box(row.RD_X, row.RD_Y, row.RD_X+50, row.RD_Y+50), axis=1)
all_tiles_gdf = gpd.GeoDataFrame(all_tiles_gdf)

In [ ]:
drop_folders = ['nl-amsd-200625-7415-laz']
all_tiles_gdf = all_tiles_gdf[~all_tiles_gdf['folder'].isin(drop_folders)]

In [ ]:
all_tiles_gdf = all_tiles_gdf[all_tiles_gdf['n_points'] >= 10000]

In [ ]:
all_tiles_gdf['naam'] = all_tiles_gdf['folder'].replace(cyclomedia_dict)

In [ ]:
stadsdelen_tiles = all_tiles_gdf[['naam', 'geometry']].dissolve(by='naam')

In [ ]:
all_tiles_gdf.to_file('run2_tiles_10k.gpkg', driver='GPKG')

## Plot data

In [ ]:
fig, ax = plt.subplots(figsize=(8,6))

stadsdelen_tiles.plot(ax=ax, cmap='tab10', alpha=0.5, legend=True)
stadsdelen_poly.boundary.plot(ax=ax, edgecolor='black')
# stadsdelen_poly.plot(ax=ax, column='district_code', cmap='tab10', legend=True)

In [ ]:
plt.savefig('stadsdelen_dekking.png', ax=ax, dpi=200)

## Generate BGT per stadsdeel

In [ ]:
stadsdelen_poly.reset_index()

In [ ]:
for _, row in stadsdelen_poly.reset_index().iterrows():
    bgt_folder = f"{stadsdelen_folder}{row['naam']}/"
    target_area = row['geometry']

    (x_min, y_min, x_max, y_max) = target_area.bounds
    bbox = ((x_min, y_max), (x_max, y_min))

    # Output file
    bgt_data_file = f'{bgt_folder}bgt_voetpad.gpkg'
    terras_data_file = f'{bgt_folder}terras_data.gpkg'
    obstacle_data_file = f'{bgt_folder}obstacle_data.gpkg'

    # Voetpad
    bgt_layers = ['BGT_WGL_voetpad']
    gdf = scraping.get_bgt_data_for_bbox(bbox, bgt_layers)
    gdf = gdf[gdf.intersects(target_area)]
    if len(gdf) > 0:
        gdf.to_file(bgt_data_file, driver='GPKG')

    # Terras
    gdf = scraping.get_terras_data_for_bbox(bbox)
    gdf = gdf[gdf.intersects(target_area)]
    if len(gdf) > 0:
        gdf.to_file(terras_data_file, driver='GPKG')

    # Obstacles
    bgt_layers = ['BGTPLUS_VGT_boom']
    gdf = scraping.get_bgt_data_for_bbox(bbox, bgt_layers)
    gdf = gdf[gdf.intersects(target_area)]
    if len(gdf) > 0:
        gdf.to_file(obstacle_data_file, driver='GPKG')